In [1]:
from bs4 import BeautifulSoup


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bSoup = BeautifulSoup(data, 'html.parser')
#bSoup

In [3]:
postalCodeList = []
boroughList = []
neighborList = []

for row in bSoup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborList.append(cells[2].text.rstrip('\n'))

In [4]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
toronto_info = [('PostalCode',postalCodeList), ('Borough', boroughList),('Neighbourhood', neighborList)]
toronto_df = pd.DataFrame.from_dict(dict(toronto_info))
toronto_df.head(20)

,Borough,Neighbourhood,PostalCode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
9,Not assigned,Not assigned,M8A


In [5]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
toronto_df_filtered = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_df_filtered.head(20)

,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A
5,North York,Lawrence Manor,M6A
6,Queen's Park,Not assigned,M7A
7,Etobicoke,Islington Avenue,M9A
8,Scarborough,Rouge,M1B
9,Scarborough,Malvern,M1B


In [6]:
#append comma and sort/group based on postal code and  Borough
toronto_df_joined = toronto_df_filtered.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: ','.join(x))
toronto_df_joined.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [7]:
# use the .shape method to print the number of rows of your dataframe.
toronto_df_joined.shape

(103, 3)

In [8]:
#using the e Geocoder package
url="http://cocl.us/Geospatial_data"
coordinates = pd.read_csv(url)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
toronto_df_join = toronto_df_joined.set_index('PostalCode')
coordinates_join = coordinates.set_index('Postal Code')
toronto_df_coordinates = pd.concat([toronto_df_join, coordinates_join], axis=1, join='inner')
toronto_df_coordinates.index.name = 'PostalCode'

toronto_df_coordinates.reset_index(inplace=True)
toronto_df_coordinates.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [10]:
toronto_df_coordinates.shape

(103, 5)